# 셀레니움으로 자료 가져오기
* chrome 웹드라이버설치 [설치사이트]( https://chromedriver.chromium.org/downloads)
* 셀레니움참고 [참고사이트](https://galid1.tistory.com/478)

In [2]:
# #선행설치
# !pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options

import time

import pandas as pd
import numpy as np

import os

# 본문내용긁어오기

In [4]:
#저장위치
final_dict = dict()
final_dict['ho'] = []
final_dict['dates'] = []
final_dict['category'] = []
final_dict['city'] = []
final_dict['title'] = []
final_dict['contents'] = []

def get_contents():    
    '''
    '호'를 누르고 난 다음 내부의 본문들을 클릭하여 들어감 
    '''
    try :
        for j in range(2,100): #최대길이는 50으로 추정되나 100으로 두어도 무리없을것으로 보임 
            one_title = driver.find_element_by_xpath('//*[@id="container_suite"]/div[4]/div/div/ul/li[{num}]/div[1]/h2/a'.format(num =j))
            one_title.click()

            article_xpath = driver.find_element_by_xpath('//*[@id="center"]/article')

            #facebook, Twitter의 위치가 계속 변동되어 지우고 위치지정
            article = article_xpath.text.split('\n')
            article.remove('Facebook')
            article.remove('Twitter')

            #추출
            title = article[0]
            dates = article[2]
            ho = article[6]
            category = article[8]
            contents = article[9:-1]

            city = title.split(" ")[-1]
            if ')' in city:
                city = city.replace('(',"").replace(')',"")
            else :
                city = np.nan

            #저장 
            final_dict.get('dates',0).append(dates)
            final_dict.get('ho',0).append(ho)
            final_dict.get('category',0).append(category)
            final_dict.get('city',0).append(city)
            final_dict.get('title',0).append(title)
            final_dict.get('contents',0).append(contents)

            #뒤로가기
            time.sleep(0.5)
            driver.back()
    
    except Exception as e:
        pass 


In [5]:
def save_reset():
    
    global final_dict
    #csv 저장
    final_df = pd.DataFrame(final_dict)

    last_ho = final_df.ho.iloc[0]
    first_ho = final_df.ho.iloc[-1]

    final_df.to_csv('./result/All_document({f}-{l}).csv'.format(f = first_ho, l = last_ho))
    
    #변수초기화
    final_dict = dict()
    final_dict['ho'] = []
    final_dict['dates'] = []
    final_dict['category'] = []
    final_dict['city'] = []
    final_dict['title'] = []
    final_dict['contents'] = []

### 특정페이지만

In [12]:
#전체 진행 (오류나면서 끝나는것이 맞다)
os.chdir('D:/Python/SDI 셀레니움/')
options = Options()
# chrome을 전체화면으로/ headless
options.add_argument('--start-maximized')
# options.add_argument('--headless')
driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)

#사이트이동 원하는 페이지를 적어서 이동가능
num = 1
# driver.get('https://www.si.re.kr/world_trends/22383?paged={page_num}'.format(page_num = num))
driver.get('http://www.sdi.re.kr/world_trends/15304?paged=48') #1호 페이지 
time.sleep(2)

#1페이지 내 다운로드 
try :
    for i in range(1,11):
        #처음페이지는 9개인데 이후 10개씩
#         ho = driver.find_element_by_xpath('//*[@id="container_suite"]/div[3]/ul/li[{num}]/a'.format(num = i))
#         ho.click()
#         time.sleep(1)
        get_contents()
    save_reset()
except :
    pass
driver.quit()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys
